In [1]:
import os

In [2]:
%pwd

'c:\\Users\\acer\\Desktop\\collage project\\Workspace\\First-ml-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\acer\\Desktop\\collage project\\Workspace\\First-ml-project'

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [25]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [21]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [22]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [23]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-04 14:39:03,803: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-04 14:39:03,813: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-04 14:39:03,826: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-04 14:39:03,826: INFO: common: created directory at: artifacts]
[2024-07-04 14:39:03,834: INFO: common: created directory at: artifacts/data_ingestion]


[2024-07-04 14:39:05,489: INFO: 1268487314: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f1df60c1d3356bf6a854d12fe866c8918ef097b8805f8fe7a610b4a6548a7918"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DE11:3B565:19C20B:20DACC:668666B0
Accept-Ranges: bytes
Date: Thu, 04 Jul 2024 09:09:05 GMT
Via: 1.1 varnish
X-Served-By: cache-del21721-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1720084145.173347,VS0,VE298
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 4eaa03175ab761d98a53f5a44613419eafedc4dc
Expires: Thu, 04 Jul 2024 09:14:05 GMT
Source-Age: 0

]
